In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load the dataset
data_path = 'F:\\Masters\\Thesis\\BanglaEmotion\\Train.csv'  # Update with your path
val_data_path = 'F:\\Masters\\Thesis\\BanglaEmotion\\Val.csv'  # Validation data path
test_data_path = 'F:\\Masters\\Thesis\\BanglaEmotion\\Test.csv'  # Test data path

# Load all datasets
df = pd.read_csv(data_path)
df_val = pd.read_csv(val_data_path)
df_test = pd.read_csv(test_data_path)

# Combine emotion columns into labels for training, validation, and test data
emotion_columns = ['Love', 'Joy', 'Surprise', 'Anger', 'Sadness', 'Fear']
df['Emotion_Label'] = df[emotion_columns].idxmax(axis=1)  # Finds the emotion with the highest value
df_val['Emotion_Label'] = df_val[emotion_columns].idxmax(axis=1)
df_test['Emotion_Label'] = df_test[emotion_columns].idxmax(axis=1)

# Encode labels for emotion and topic
emotion_label_encoder = LabelEncoder()
df['Emotion_Label'] = emotion_label_encoder.fit_transform(df['Emotion_Label'])
df_val['Emotion_Label'] = emotion_label_encoder.transform(df_val['Emotion_Label'])
df_test['Emotion_Label'] = emotion_label_encoder.transform(df_test['Emotion_Label'])

topic_label_encoder = LabelEncoder()
df['Topic'] = topic_label_encoder.fit_transform(df['Topic'])
df_val['Topic'] = topic_label_encoder.transform(df_val['Topic'])
df_test['Topic'] = topic_label_encoder.transform(df_test['Topic'])

# Tokenization and Padding for Text Data
max_words = 5000  # Maximum number of words to consider
max_len = 100  # Maximum sequence length
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['Data'])

X_train = pad_sequences(tokenizer.texts_to_sequences(df['Data']), maxlen=max_len)
X_val = pad_sequences(tokenizer.texts_to_sequences(df_val['Data']), maxlen=max_len)
X_test = pad_sequences(tokenizer.texts_to_sequences(df_test['Data']), maxlen=max_len)

# Convert labels to categorical
y_train_emotion = to_categorical(df['Emotion_Label'])
y_val_emotion = to_categorical(df_val['Emotion_Label'])
y_test_emotion = to_categorical(df_test['Emotion_Label'])

y_train_topic = to_categorical(df['Topic'])
y_val_topic = to_categorical(df_val['Topic'])
y_test_topic = to_categorical(df_test['Topic'])

# CNN Model
embedding_dim = 128
filters = 128
kernel_size = 5
dropout_rate = 0.5

# Input Layer
input_layer = Input(shape=(max_len,))

# Embedding Layer
embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(input_layer)

# CNN Layer
conv_layer = Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(embedding_layer)
pooling_layer = GlobalMaxPooling1D()(conv_layer)
dropout_layer = Dropout(dropout_rate)(pooling_layer)

# Dense layers for separate outputs
emotion_output = Dense(y_train_emotion.shape[1], activation='softmax', name='emotion_output')(dropout_layer)
topic_output = Dense(y_train_topic.shape[1], activation='softmax', name='topic_output')(dropout_layer)

# Model
model = Model(inputs=input_layer, outputs=[emotion_output, topic_output])
model.compile(
    optimizer='adam',
    loss={'emotion_output': 'categorical_crossentropy', 'topic_output': 'categorical_crossentropy'},
    metrics={'emotion_output': 'accuracy', 'topic_output': 'accuracy'}
)

# Model Summary
model.summary()

# Training the Model
history = model.fit(
    X_train,
    {'emotion_output': y_train_emotion, 'topic_output': y_train_topic},
    validation_data=(X_val, {'emotion_output': y_val_emotion, 'topic_output': y_val_topic}),
    epochs=10,
    batch_size=32
)

# Evaluate on Test Data
test_loss, test_emotion_loss, test_topic_loss, test_emotion_acc, test_topic_acc = model.evaluate(
    X_test,
    {'emotion_output': y_test_emotion, 'topic_output': y_test_topic}
)

print(f"Test Emotion Accuracy: {test_emotion_acc:.4f}")
print(f"Test Topic Accuracy: {test_topic_acc:.4f}")

# Predictions
emotion_pred = np.argmax(model.predict(X_test)[0], axis=1)
topic_pred = np.argmax(model.predict(X_test)[1], axis=1)

# Classification Reports
print("Emotion Classification Report on Test Data:")
print(classification_report(np.argmax(y_test_emotion, axis=1), emotion_pred, target_names=emotion_label_encoder.classes_))

print("Topic Classification Report on Test Data:")
print(classification_report(np.argmax(y_test_topic, axis=1), topic_pred, target_names=topic_label_encoder.classes_))


E:\python\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 100)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding (Embedding)         │ (None, 100, 128)          │         640,000 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, 96, 128)           │          82,048 │ embedding[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ global_max_pooling1d          │ (None, 128)               │               0 │ conv1d[0][0]               │
│ (GlobalMaxPooling1D)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ global_max_pooling1d[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ emotion_output (Dense)        │ (None, 6)                 │             774 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ topic_output (Dense)          │ (None, 12)                │           1,548 │ dropout[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 724,370 (2.76 MB)

 Trainable params: 724,370 (2.76 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
576/576 ━━━━━━━━━━━━━━━━━━━━ 40s 46ms/step - emotion_output_accuracy: 0.3759 - emotion_output_loss: 1.4669 - loss: 3.4548 - topic_output_accuracy: 0.2894 - topic_output_loss: 1.9878 - val_emotion_output_accuracy: 0.5427 - val_emotion_output_loss: 1.1135 - val_loss: 2.5798 - val_topic_output_accuracy: 0.5305 - val_topic_output_loss: 1.4661
Epoch 2/10
576/576 ━━━━━━━━━━━━━━━━━━━━ 25s 43ms/step - emotion_output_accuracy: 0.5955 - emotion_output_loss: 1.0474 - loss: 2.4175 - topic_output_accuracy: 0.5701 - topic_output_loss: 1.3701 - val_emotion_output_accuracy: 0.5892 - val_emotion_output_loss: 1.0291 - val_loss: 2.3226 - val_topic_output_accuracy: 0.5804 - val_topic_output_loss: 1.2932
Epoch 3/10
576/576 ━━━━━━━━━━━━━━━━━━━━ 24s 42ms/step - emotion_output_accuracy: 0.6632 - emotion_output_loss: 0.8899 - loss: 2.0004 - topic_output_accuracy: 0.6476 - topic_output_loss: 1.1106 - val_emotion_output_accuracy: 0.5794 - val_emotion_output_loss: 1.0444 - val_loss: 2.3373 - val_topic_

E:\python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\python\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
